In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as optimization
from tqdm.notebook import tqdm
import utilities as uts
#from BHDVCStf import BHDVCS
from TVA1_UU import TVA1_UU #modified bhdvcs file
import matplotlib.pyplot as plt
from iminuit import Minuit

In [4]:
filename="dvcs_xs_May-2021_342_sets.csv"
numSets = 10
numReplicas = 1

In [32]:
#####
class DvcsData(object):
    def __init__(self, df):
        self.df=df.copy()
        self.X = df.loc[:, ['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'dvcs']]
        self.XnoCFF = df.loc[:, ['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'dvcs']]
        self.y = df.loc[:, 'F']
        self.Kinematics = df.loc[:, ['k', 'QQ', 'x_b', 't']]
        self.erry = df.loc[:, 'sigmaF']

    def __len__(self):
        return len(self.X)

    def getSet(self, setNum, itemsInSet=45):
        pd.options.mode.chained_assignment = None
        subX = self.X.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1, :]
        subX['F'] = self.y.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1]
        subX['sigmaF'] = self.erry.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1]
        pd.options.mode.chained_assignment = 'warn'
        return DvcsData(subX)

    def sampleX(self):
        #return np.random.normal(self.y, self.erry)
        return self.X    
    
    def sampleY(self):
        #return np.random.normal(self.y, self.erry)
        return self.y

    def sampleErrY(self):
        #return np.random.normal(self.y, self.erry)
        return self.erry    
    
    def sampleWeights(self):
        return 1/self.erry
    
    
bhdvcs = TVA1_UU()
df = pd.read_csv(filename)
data = DvcsData(df)
XX = np.array(data.sampleX())
YY = np.array(data.sampleY())
eYY = np.array(data.sampleErrY())

def SelectSet(setnum):
    tempdf=pd.read_csv(filename)
    full_data_size=len(tempdf["#Set"])
    temp_k=[]
    temp_QQ=[]
    temp_xb=[]
    temp_t=[]
    temp_phi=[]
    temp_F=[]
    temp_dF=[]
    print(full_data_size)
    for i in range(0,full_data_size):
        if(tempdf["#Set"][i]==setnum):
            temp_k.append(tempdf["k"][i])
            temp_phi.append(tempdf["phi_x"][i])
    return temp_phi

def TotalChi2XS(ReH, ReE, ReHtilde):
    tempTheory=bhdvcs.TotalUUXS(XX, ReH, ReE, ReHtilde)
    tempY=data.sampleY()
    tempYErr=data.erry
    return np.sum(((YY-tempTheory)/eYY)**2)
    #return tempTheory
     

#print(XX[0])
print(TotalChi2XS(1,1,1))

29363.293367840455


In [38]:
def num1(x):
   def num2():
      return 2*x 
   return num2
res = num1(10)

print(res())

20


In [96]:
def Indv_Set(i):
    SetI = data.getSet(i)
    X  = np.array(SetI.XnoCFF)
    Y  = SetI.sampleY()
    eY = SetI.erry
    #pars = np.array([1, 1, 1])
    def TotalChi2XS(ReH, ReE, ReHtilde):
        tempTheory=bhdvcs.TotalUUXS(X, ReH, ReE, ReHtilde)
        return np.sum(((Y-tempTheory)/eY)**2)
    m = Minuit(TotalChi2XS,0,0,0)
    m.migrad()
    tempCFFs = m.values
    tempErrs = m.errors
    return (i,tempCFFs[0],tempErrs[0],tempCFFs[1],tempErrs[1],tempCFFs[2],tempErrs[2])

testfitset = Indv_Set(0)
print(testfitset)  
print(int(len(XX)/45))

<ipython-input-96-68c7987896c6>:11: IMinuitWarning: errordef not set, using 1 (appropriate for least-squares)
  m.migrad()


(0, 0.702109542882553, 0.6331402940281065, -2.9586949200069057, 27.04562249153774, 3.497793365959005, 1.281142898247682)
342


In [90]:
# {"#Set":[],"ReH":[],"sigmaReH":[],"ReE":[],"sigmaReE":[],"ReHtilde":[],"sigmaReHtilde":[]}
# columns=["#Set","ReH","sigmaReH","ReE","sigmaReE","ReHtilde","sigmaReHtilde"]

results_df = pd.DataFrame()

testresult=[]
setn=[]
tst_ReH=[]
tst_ReH_err=[]
tst_ReE=[]
tst_ReE_err=[]
tst_ReHtilde=[]
tst_ReHtilde_err=[]

for i in range(5):
    testfit = Indv_Set(i)
    setn.append(int(np.array(testfit)[0]))
    tst_ReH.append(np.array(testfit)[1])
    tst_ReH_err.append(np.array(testfit)[2])
    tst_ReE.append(np.array(testfit)[3])
    tst_ReE_err.append(np.array(testfit)[4])
    tst_ReHtilde.append(np.array(testfit)[5])
    tst_ReHtilde_err.append(np.array(testfit)[6])
    print("Completed fitting to data set #",i)
    #results_df.append({"#Set":np.array(testfit)[0],"ReH":np.array(testfit)[1],"sigmaReH":np.array(testfit)[2],"ReE":np.array(testfit)[3],"sigmaReE":np.array(testfit)[4],"ReHtilde":np.array(testfit)[5],"sigmaReHtilde":np.array(testfit)[6]}, ignore_index=True)
    #print(np.array(testfit)[1])
#     results_df["#Set"]=np.array(testfit)[0]
#     results_df["ReH"]=np.array(testfit)[1]
#     results_df["sigmaReH"]=np.array(testfit)[2]
#     results_df["ReE"]=np.array(testfit)[3]
#     results_df["sigmaReE"]=np.array(testfit)[4]
#     results_df["ReHtilde"]=np.array(testfit)[5]
#     results_df["sigmaReHtilde"]=np.array(testfit)[6]
#     testresult.append(results_df)
    #print(testresult)
    #resultsI=pd.concat(testresult)

    
results_df["#Set"]=setn
results_df["ReH"]=tst_ReH
results_df["sigmaReH"]=tst_ReH_err
results_df["ReE"]=tst_ReE
results_df["sigmaReE"]=tst_ReE_err
results_df["ReHtilde"]=tst_ReHtilde
results_df["sigmaReHtilde"]=tst_ReHtilde_err
testresult.append(results_df)
    
print(testresult)

#print(results_df)
fitresults = pd.concat(testresult)
#print(fitresults)
fitresults.to_csv('LocalFitResults_iminuit.csv')

<ipython-input-89-b5b294bf007b>:11: IMinuitWarning: errordef not set, using 1 (appropriate for least-squares)
  m.migrad()


Completed fitting to data set # 0
Completed fitting to data set # 1
Completed fitting to data set # 2
Completed fitting to data set # 3
Completed fitting to data set # 4
[   #Set       ReH  sigmaReH        ReE   sigmaReE  ReHtilde  sigmaReHtilde
0     0  0.702110  0.633140  -2.958695  27.045622  3.497793       1.281143
1     1  1.368832  1.574534  -1.575320  43.714407  5.507605       9.187427
2     2  1.839356  5.451255  -2.552745  49.284926  5.174919      30.530259
3     3  1.218489  4.583288   3.287532  49.443313  3.230697      14.965805
4     4  1.142197  2.551233  12.107739  36.527655 -3.552481      20.242958]


In [6]:
def produceResults(data, numSets, numReplicas):
    {"#Set":[],"k":[],"QQ":[],"x_b":[],"t":[],"phi_x":[],"F_data":[],"sigmaF_data":[],"F_cal":[],"sigmaF_cal":[],"ReH":[],"sigmaReH":[],"ReE":[],"sigmaReE":[],"ReHtilde":[],"sigmaReHtilde":[]}
    results = []
    print("Here we start")
    for i in tqdm(range(numSets)):
        CFFreplicas = []
        Temp_Fcals=[]
        Temp_H=[]
        Temp_E=[]
        Temp_Htilde=[]
        Temp_phi=[]
        seti = data.getSet(i) # DvcsData object of kinematic set i
        X = np.array(seti.XnoCFF) # the kinematics and all variables necessary to compute 
        y = seti.sampleY()
        sigma = seti.erry # error in F
        pars = np.array([1, 1, 1])
        Tempdf=seti.df.copy()
        #print(TotalChi2XS(seti, X, 1, 1, 1))
        m = Minuit(TotalChi2XS, seti)
        m.migrad()
        #cff, cffcov = optimization.curve_fit(bhdvcs.TotalUUXS, X, y, pars, sigma,method='lm')
        print(m.values)
        print(m.errors)
#         for j in range(numReplicas):
#             y = seti.sampleY()
#             cff, cffcov = optimization.curve_fit(bhdvcs.TotalUUXS, X, y, pars, sigma,method='lm')
#             Temp_Fcals.append(bhdvcs.TotalUUXS(np.array(data.getSet(i).X),cff[0],cff[1],cff[2]))
#             Temp_H.append(cff[0])
#             Temp_E.append(cff[1])
#             Temp_Htilde.append(cff[2])
#         Tempdf["F_cal"]=np.mean(np.array(Temp_Fcals),axis=0)
#         Tempdf["sigmaF_cal"]=np.std(np.array(Temp_Fcals),axis=0)
#         Tempdf["ReH"]=np.mean(Temp_H)
#         Tempdf["sigmaReH"]=np.std(Temp_H)
#         Tempdf["ReE"]=np.mean(Temp_E)
#         Tempdf["sigmaReE"]=np.std(Temp_E)
#         Tempdf["ReHtilde"]=np.mean(Temp_Htilde)
#         Tempdf["sigmaReHtilde"]=np.std(Temp_Htilde)
#         Tempdf["#Set"]=i
#         results.append(Tempdf)
#     return pd.concat(results)



produceResults(data, numSets, numReplicas)

Here we start


RuntimeError: 1 values given for 5 function parameter(s)